

In this notebook, you will:
1. **Understand** the Frozen Lake environment and reinforcement learning basics
2. **Implement** a Deep Q-Network (DQN) from scratch using PyTorch
3. **Train** the neural network to solve the Frozen Lake problem
4. **Visualize** the learning process and agent's performance
5. **Analyze** the trained policy and Q-values

Frozen Lake is a classic reinforcement learning environment where:
- **Goal**: Navigate from start (S) to goal (G) on a frozen lake
- **Challenge**: Avoid falling into holes (H) while walking on frozen tiles (F)
- **Complexity**: The lake is slippery - you don't always move in the intended direction!

```
SFFF    S = Start
FHFH    F = Frozen (safe)
FFFH    H = Hole (game over)
HFFG    G = Goal (success!)
```


Q-Learning learns the **Q-function**: Q(s,a) = expected future reward when taking action 'a' in state 's'.

- **Traditional Q-Learning**: Uses a table to store Q-values
- **Deep Q-Learning**: Uses a neural network to approximate the Q-function
- **Advantage**: Can handle large or continuous state spaces

1. **Neural Network**: Approximates Q(s,a) for all actions given a state
2. **Experience Replay**: Stores and randomly samples past experiences
3. **Target Network**: Stabilizes training
4. **ε-greedy Policy**: Balances exploration vs exploitation

```
Q(s,a) = r + γ * max(Q(s',a'))
```


In [ ]:

!pip install gymnasium[toy-text] torch torchvision matplotlib seaborn numpy pandas tqdm

import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import deque, namedtuple
import random
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

plt.style.use('default')
sns.set_palette("husl")

print("✅ All packages installed and imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🏋️ Gymnasium version: {gym.__version__}")


In [ ]:

env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True, render_mode='rgb_array')

print("🏔️ Frozen Lake Environment Information:")
print(f"📊 Observation Space: {env.observation_space}")
print(f"🎮 Action Space: {env.action_space}")
print(f"🗺️ Map Size: 4x4 = 16 states")
print(f"🎯 Actions: 0=Left, 1=Down, 2=Right, 3=Up")

print("\n🗺️ Map Layout:")
desc = env.unwrapped.desc
for i, row in enumerate(desc):
    if hasattr(row[0], 'decode'):
        row_str = ' '.join([cell.decode('utf-8') for cell in row])
    else:
        row_str = ' '.join([str(cell) for cell in row])
    print(f"Row {i}: {row_str}")

state, info = env.reset()
print(f"\n🚀 Initial state: {state}")
print(f"📍 Initial position: Row {state // 4}, Column {state % 4}")


In [ ]:

def visualize_environment(env, state=None, title="Frozen Lake Environment"):
    """Visualize the Frozen Lake environment with the agent's current position."""
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    desc = env.unwrapped.desc
    nrows, ncols = desc.shape
    
    colors = {
        b'S': 'lightgreen',  # Start
        b'F': 'lightblue',   # Frozen
        b'H': 'red',         # Hole
        b'G': 'gold'         # Goal
    }
    
    for i in range(nrows):
        for j in range(ncols):
            tile = desc[i, j]
            color = colors[tile]
            
            rect = plt.Rectangle((j, nrows-1-i), 1, 1, 
                               facecolor=color, edgecolor='black', linewidth=2)
            ax.add_patch(rect)
            
            ax.text(j+0.5, nrows-1-i+0.5, tile.decode('utf-8') if hasattr(tile, 'decode') else str(tile), 
                   ha='center', va='center', fontsize=20, fontweight='bold')
    
    if state is not None:
        agent_row = state // ncols
        agent_col = state % ncols
        
        circle = plt.Circle((agent_col+0.5, nrows-1-agent_row+0.5), 0.3, 
                          color='purple', alpha=0.8, zorder=10)
        ax.add_patch(circle)
        
        ax.text(agent_col+0.5, nrows-1-agent_row+0.5, '🤖', 
               ha='center', va='center', fontsize=16, zorder=11)
    
    ax.set_xlim(0, ncols)
    ax.set_ylim(0, nrows)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='lightgreen', label='Start (S)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightblue', label='Frozen (F)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='red', label='Hole (H)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='gold', label='Goal (G)'),
        plt.Circle((0, 0), 0.1, color='purple', label='Agent 🤖')
    ]
    ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.tight_layout()
    plt.show()

visualize_environment(env, state, "🏔️ Frozen Lake - Initial State")


In [ ]:

class DQN(nn.Module):
    """Deep Q-Network for Frozen Lake environment."""
    
    def __init__(self, state_size=16, action_size=4, hidden_size=128):
        super(DQN, self).__init__()
        
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, action_size)
        
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        if x.dim() == 1 and x.dtype == torch.long:
            x = F.one_hot(x, num_classes=16).float()
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

dqn = DQN().to(device)
print(f"\n🧠 DQN Architecture:")
print(dqn)

test_state = torch.tensor([0], dtype=torch.long).to(device)
test_output = dqn(test_state)
print(f"\n🧪 Test input (state 0): {test_state}")
print(f"🎯 Test output (Q-values): {test_output}")
print(f"📊 Network parameters: {sum(p.numel() for p in dqn.parameters())}")


In [ ]:

Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """Experience Replay Buffer for storing and sampling experiences."""
    
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
    
    def push(self, state, action, reward, next_state, done):
        experience = Experience(state, action, reward, next_state, done)
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        experiences = random.sample(self.buffer, batch_size)
        
        states = torch.tensor([e.state for e in experiences], dtype=torch.long).to(device)
        actions = torch.tensor([e.action for e in experiences], dtype=torch.long).to(device)
        rewards = torch.tensor([e.reward for e in experiences], dtype=torch.float32).to(device)
        next_states = torch.tensor([e.next_state for e in experiences], dtype=torch.long).to(device)
        dones = torch.tensor([e.done for e in experiences], dtype=torch.bool).to(device)
        
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return len(self.buffer)

print("📦 Replay Buffer implemented successfully!")


In [ ]:

class DQNAgent:
    """Deep Q-Network Agent for solving Frozen Lake."""
    
    def __init__(self, state_size=16, action_size=4, lr=0.001, gamma=0.99, 
                 epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995,
                 buffer_size=10000, batch_size=32, target_update=100):
        
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update = target_update
        
        self.q_network = DQN(state_size, action_size).to(device)
        self.target_network = DQN(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        
        self.target_network.load_state_dict(self.q_network.state_dict())
        
        self.memory = ReplayBuffer(buffer_size)
        
        self.training_step = 0
        self.losses = []
        
    def act(self, state, training=True):
        """Choose an action using ε-greedy policy."""
        if training and random.random() < self.epsilon:
            return random.randrange(self.action_size)
        
        state_tensor = torch.tensor([state], dtype=torch.long).to(device)
        q_values = self.q_network(state_tensor)
        return q_values.argmax().item()
    
    def remember(self, state, action, reward, next_state, done):
        """Store experience in replay buffer."""
        self.memory.push(state, action, reward, next_state, done)
    
    def replay(self):
        """Train the network on a batch of experiences."""
        if len(self.memory) < self.batch_size:
            return
        
        states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)
        
        current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1))
        next_q_values = self.target_network(next_states).max(1)[0].detach()
        target_q_values = rewards + (self.gamma * next_q_values * ~dones)
        
        loss = F.mse_loss(current_q_values.squeeze(), target_q_values)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.losses.append(loss.item())
        
        self.training_step += 1
        if self.training_step % self.target_update == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def get_q_values(self, state):
        """Get Q-values for a given state."""
        state_tensor = torch.tensor([state], dtype=torch.long).to(device)
        with torch.no_grad():
            return self.q_network(state_tensor).cpu().numpy()[0]

agent = DQNAgent()
print("🤖 DQN Agent created successfully!")
print(f"🧠 Network parameters: {sum(p.numel() for p in agent.q_network.parameters())}")
print(f"🎯 Initial epsilon (exploration rate): {agent.epsilon:.3f}")
print(f"💾 Replay buffer capacity: {agent.memory.capacity}")


In [ ]:

def train_dqn(agent, env, episodes=2000, max_steps=100, print_every=200):
    """Train the DQN agent."""
    scores = []
    success_rate = []
    epsilon_history = []
    recent_scores = deque(maxlen=100)
    
    print("🚀 Starting DQN Training...")
    print(f"📊 Episodes: {episodes}, Max steps per episode: {max_steps}")
    print("=" * 60)
    
    for episode in tqdm(range(episodes), desc="Training Progress"):
        state, _ = env.reset()
        total_reward = 0
        
        for step in range(max_steps):
            action = agent.act(state, training=True)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            agent.remember(state, action, reward, next_state, done)
            agent.replay()
            
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        scores.append(total_reward)
        recent_scores.append(total_reward)
        epsilon_history.append(agent.epsilon)
        
        if len(recent_scores) >= 100:
            success_rate.append(sum(recent_scores) / len(recent_scores))
        else:
            success_rate.append(sum(recent_scores) / len(recent_scores))
        
        if (episode + 1) % print_every == 0:
            avg_score = np.mean(recent_scores)
            print(f"Episode {episode + 1:4d} | Avg Score: {avg_score:.3f} | Success Rate: {success_rate[-1]:.3f} | Epsilon: {agent.epsilon:.3f} | Buffer Size: {len(agent.memory)}")
    
    print("\n✅ Training completed!")
    
    return {
        'scores': scores,
        'success_rate': success_rate,
        'epsilon_history': epsilon_history,
        'losses': agent.losses
    }

training_stats = train_dqn(agent, env, episodes=2000, max_steps=100, print_every=200)

print(f"\n🎉 Final training statistics:")
print(f"📈 Final success rate: {training_stats['success_rate'][-1]:.3f}")
print(f"🎯 Final epsilon: {training_stats['epsilon_history'][-1]:.3f}")
print(f"💾 Total experiences collected: {len(agent.memory)}")
print(f"🔄 Training steps completed: {agent.training_step}")


In [ ]:

def plot_training_results(stats):
    """Plot comprehensive training results."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('🧠 DQN Training Results - Frozen Lake', fontsize=16, fontweight='bold')
    
    axes[0, 0].plot(stats['success_rate'], color='green', linewidth=2)
    axes[0, 0].set_title('📈 Success Rate Over Time')
    axes[0, 0].set_xlabel('Episode')
    axes[0, 0].set_ylabel('Success Rate')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='Target (80%)')
    axes[0, 0].legend()
    
    axes[0, 1].plot(stats['epsilon_history'], color='orange', linewidth=2)
    axes[0, 1].set_title('🎯 Exploration Rate (Epsilon) Decay')
    axes[0, 1].set_xlabel('Episode')
    axes[0, 1].set_ylabel('Epsilon')
    axes[0, 1].grid(True, alpha=0.3)
    
    window_size = 50
    if len(stats['scores']) >= window_size:
        smoothed_scores = pd.Series(stats['scores']).rolling(window=window_size).mean()
        axes[1, 0].plot(smoothed_scores, color='blue', linewidth=2, label=f'Smoothed (window={window_size})')
    
    axes[1, 0].plot(stats['scores'], color='lightblue', alpha=0.3, label='Raw scores')
    axes[1, 0].set_title('🏆 Episode Scores')
    axes[1, 0].set_xlabel('Episode')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    
    if stats['losses']:
        loss_window = min(100, len(stats['losses']) // 10)
        if len(stats['losses']) >= loss_window:
            smoothed_loss = pd.Series(stats['losses']).rolling(window=loss_window).mean()
            axes[1, 1].plot(smoothed_loss, color='red', linewidth=2)
        
        axes[1, 1].set_title('📉 Training Loss')
        axes[1, 1].set_xlabel('Training Step')
        axes[1, 1].set_ylabel('MSE Loss')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Training Summary:")
    print(f"🎯 Final Success Rate: {stats['success_rate'][-1]:.1%}")
    print(f"📈 Best Success Rate: {max(stats['success_rate']):.1%}")
    print(f"🏆 Total Successful Episodes: {sum(stats['scores'])}")
    print(f"📉 Final Exploration Rate: {stats['epsilon_history'][-1]:.3f}")
    
    if stats['losses']:
        print(f"🔥 Final Training Loss: {stats['losses'][-1]:.6f}")
        print(f"📊 Average Training Loss: {np.mean(stats['losses']):.6f}")

plot_training_results(training_stats)


In [ ]:

def test_agent(agent, env, num_episodes=100):
    """Test the trained agent's performance."""
    test_scores = []
    test_steps = []
    successful_episodes = 0
    
    print(f"🧪 Testing agent for {num_episodes} episodes...")
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        total_reward = 0
        steps = 0
        
        for step in range(100):
            action = agent.act(state, training=False)
            next_state, reward, terminated, truncated, _ = env.step(action)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if terminated or truncated:
                break
        
        test_scores.append(total_reward)
        test_steps.append(steps)
        
        if total_reward > 0:
            successful_episodes += 1
    
    success_rate = successful_episodes / num_episodes
    avg_score = np.mean(test_scores)
    avg_steps = np.mean(test_steps)
    
    print(f"\n🎯 Test Results:")
    print(f"✅ Success Rate: {success_rate:.1%} ({successful_episodes}/{num_episodes})")
    print(f"🏆 Average Score: {avg_score:.3f}")
    print(f"👣 Average Steps: {avg_steps:.1f}")
    print(f"🎲 Score Distribution: {np.bincount(test_scores)}")
    
    return {
        'success_rate': success_rate,
        'scores': test_scores,
        'steps': test_steps,
        'successful_episodes': successful_episodes
    }

test_results = test_agent(agent, env, num_episodes=1000)


In [ ]:

def analyze_q_values(agent, env):
    """Analyze and visualize the learned Q-values."""
    q_table = np.zeros((16, 4))
    
    for state in range(16):
        q_values = agent.get_q_values(state)
        q_table[state] = q_values
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🧠 Learned Q-Values Analysis', fontsize=16, fontweight='bold')
    
    action_names = ['Left', 'Down', 'Right', 'Up']
    sns.heatmap(q_table, annot=True, fmt='.2f', cmap='viridis',
                xticklabels=action_names, yticklabels=range(16),
                ax=axes[0, 0])
    axes[0, 0].set_title('Q-Values for All States')
    axes[0, 0].set_xlabel('Actions')
    axes[0, 0].set_ylabel('States')
    
    best_actions = np.argmax(q_table, axis=1)
    action_grid = best_actions.reshape(4, 4)
    
    sns.heatmap(action_grid, annot=True, fmt='d', cmap='Set3',
                ax=axes[0, 1], cbar_kws={'label': 'Best Action'})
    axes[0, 1].set_title('Best Action per State (0=L, 1=D, 2=R, 3=U)')
    
    axes[1, 0].hist(q_table.flatten(), bins=30, alpha=0.7, color='skyblue')
    axes[1, 0].set_title('Q-Value Distribution')
    axes[1, 0].set_xlabel('Q-Value')
    axes[1, 0].set_ylabel('Frequency')
    
    state_values = np.max(q_table, axis=1).reshape(4, 4)
    sns.heatmap(state_values, annot=True, fmt='.2f', cmap='RdYlGn',
                ax=axes[1, 1])
    axes[1, 1].set_title('State Values (Max Q-Value)')
    
    plt.tight_layout()
    plt.show()
    
    return q_table

q_table = analyze_q_values(agent, env)

print("\n🎯 Q-Values Analysis:")
print(f"📊 Q-value range: [{q_table.min():.3f}, {q_table.max():.3f}]")
print(f"📈 Average Q-value: {q_table.mean():.3f}")
print(f"🎲 Q-value std: {q_table.std():.3f}")


In [ ]:

def visualize_policy(agent, env):
    """Visualize the learned policy on the grid."""
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    desc = env.unwrapped.desc
    nrows, ncols = desc.shape
    
    colors = {
        b'S': 'lightgreen',  # Start
        b'F': 'lightblue',   # Frozen
        b'H': 'red',         # Hole
        b'G': 'gold'         # Goal
    }
    
    action_arrows = {
        0: '←',  # Left
        1: '↓',  # Down
        2: '→',  # Right
        3: '↑'   # Up
    }
    
    for i in range(nrows):
        for j in range(ncols):
            tile = desc[i, j]
            color = colors[tile]
            
            rect = plt.Rectangle((j, nrows-1-i), 1, 1, 
                               facecolor=color, edgecolor='black', linewidth=2)
            ax.add_patch(rect)
            
            ax.text(j+0.2, nrows-1-i+0.8, tile.decode('utf-8') if hasattr(tile, 'decode') else str(tile), 
                   ha='left', va='top', fontsize=16, fontweight='bold')
            
            state = i * ncols + j
            if tile != b'H':
                best_action = agent.act(state, training=False)
                arrow = action_arrows[best_action]
                ax.text(j+0.5, nrows-1-i+0.4, arrow, 
                       ha='center', va='center', fontsize=24, 
                       color='darkblue', fontweight='bold')
                
                q_values = agent.get_q_values(state)
                max_q = np.max(q_values)
                ax.text(j+0.8, nrows-1-i+0.2, f'{max_q:.2f}', 
                       ha='right', va='bottom', fontsize=10, 
                       color='darkred', fontweight='bold')
    
    ax.set_xlim(0, ncols)
    ax.set_ylim(0, nrows)
    ax.set_aspect('equal')
    ax.set_title('🧠 Learned Policy Visualization\n(Arrows show best actions, numbers show max Q-values)', 
                 fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='lightgreen', label='Start (S)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='lightblue', label='Frozen (F)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='red', label='Hole (H)'),
        plt.Rectangle((0, 0), 1, 1, facecolor='gold', label='Goal (G)')
    ]
    ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.tight_layout()
    plt.show()

visualize_policy(agent, env)


In [ ]:

def demonstrate_agent(agent, env, num_demos=5):
    """Demonstrate the trained agent playing the game."""
    print("🎮 Agent Demonstration:")
    print("=" * 50)
    
    action_names = ['Left', 'Down', 'Right', 'Up']
    
    for demo in range(num_demos):
        print(f"\n🎯 Demo {demo + 1}:")
        state, _ = env.reset()
        total_reward = 0
        path = [state]
        actions_taken = []
        
        for step in range(20):  # Max 20 steps
            action = agent.act(state, training=False)
            next_state, reward, terminated, truncated, _ = env.step(action)
            
            actions_taken.append(action_names[action])
            path.append(next_state)
            total_reward += reward
            
            print(f"  Step {step + 1}: State {state} → Action: {action_names[action]} → State {next_state}")
            
            state = next_state
            
            if terminated or truncated:
                if reward > 0:
                    print(f"  🎉 SUCCESS! Reached goal in {step + 1} steps!")
                else:
                    print(f"  💀 Failed! Fell into hole at step {step + 1}")
                break
        
        print(f"  📊 Total reward: {total_reward}")
        print(f"  🛤️ Path: {' → '.join(map(str, path))}")
        print(f"  🎮 Actions: {' → '.join(actions_taken)}")

demonstrate_agent(agent, env, num_demos=5)




1. **🏗️ Built a Deep Q-Network** from scratch using PyTorch
2. **🎯 Implemented key DQN features**: Experience replay, target networks, ε-greedy exploration
3. **🏋️ Trained the agent** to solve the Frozen Lake environment
4. **📊 Visualized the learning process** with comprehensive plots
5. **🔍 Analyzed the learned policy** through Q-value visualization
6. **🎮 Demonstrated the trained agent** in action

- **Neural networks can learn optimal policies** even in stochastic environments
- **Experience replay** helps stabilize learning by breaking correlations
- **Target networks** provide stable learning targets
- **ε-greedy exploration** balances exploration vs exploitation

- **Traditional Q-Learning**: Uses a lookup table (16x4 = 64 entries for Frozen Lake)
- **Deep Q-Learning**: Uses a neural network that can generalize and handle larger state spaces
- **Advantage**: Better scalability and can learn complex patterns

- Try different network architectures (deeper, wider, different activations)
- Experiment with hyperparameters (learning rate, epsilon decay, etc.)
- Test on larger environments (8x8 Frozen Lake)
- Implement advanced techniques (Double DQN, Dueling DQN, Rainbow)

You've successfully implemented and trained a Deep Q-Network to solve a classic reinforcement learning problem! The neural network learned to navigate the slippery frozen lake and reach the goal while avoiding holes.

- [Deep Q-Learning Paper](https://arxiv.org/abs/1312.5602)
- [Gymnasium Documentation](https://gymnasium.farama.org/)
- [PyTorch RL Tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)
